# DataFrame2ProjectTable

Load a DataFrame of repositories and write it to a GitHub project table

In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
import requests

In [2]:
load_dotenv()

True

## Configuration

In [3]:
url = 'https://api.github.com/graphql'
token = os.getenv('ICSC_GITHUB_ORG_PAT')

org_name = "ICSC-CN-HPC-Spoke-4-Earth-Climate"
proj_num = 3

## Get project ID

In [99]:
# Query GraphQL
query = """
query ($login: String!, $number: Int!) {
  organization(login: $login) {
    projectV2(number: $number) {
      id
    }
  }
}
"""

# Intestazioni della richiesta
headers = {
    'Authorization': f'Bearer {token}',
    "Accept": "application/vnd.github.v4+json",
    'Content-Type': 'application/json'
}

# Dati della richiesta (payload)
payload = {
    "query": query,
    "variables": {
        "login": org_name,
        "number": proj_num
    }
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    print('Success:', response.json())
else:
    print('Error:', response.status_code, response.text)

Success: {'data': {'organization': {'projectV2': {'id': 'PVT_kwDOCY2ck84Aj7Zu'}}}}


In [100]:
project_id = response.json()['data']['organization']['projectV2']['id']
print(f'Project ID: {project_id}')

Project ID: PVT_kwDOCY2ck84Aj7Zu


## Get Table Fields IDs

In [101]:
query = f"""
query {{
  node(id: "{project_id}") {{
    ... on ProjectV2 {{
      fields(first: 20) {{
        nodes {{
          ... on ProjectV2Field {{
            id
            name
          }}
          ... on ProjectV2IterationField {{
            id
            name
            configuration {{
              iterations {{
                startDate
                id
              }}
            }}
          }}
          ... on ProjectV2SingleSelectField {{
            id
            name
            options {{
              id
              name
            }}
          }}
        }}
      }}
    }}
  }}
}}
"""

headers = {
    'Authorization': f'Bearer {token}',
    "Accept": "application/vnd.github.v4+json",
    'Content-Type': 'application/json'
}


data = {
    'query': query
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    print('Success:', response.json())
else:
    print('Error:', response.status_code, response.text)

Success: {'data': {'node': {'fields': {'nodes': [{'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBQ', 'name': 'Title'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBU', 'name': 'Assignees'}, {'id': 'PVTSSF_lADOCY2ck84Aj7ZuzgcNuBY', 'name': 'Status', 'options': [{'id': 'f75ad846', 'name': 'Todo'}, {'id': '47fc9ee4', 'name': 'In Progress'}, {'id': '98236657', 'name': 'Done'}]}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBc', 'name': 'Labels'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBg', 'name': 'Linked pull requests'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBk', 'name': 'Milestone'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuBo', 'name': 'Repository'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcNuB4', 'name': 'Reviewers'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIVo', 'name': 'Repository name'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIVs', 'name': 'WP'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIWg', 'name': 'Task'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIZs', 'name': 'Repository manager'}, {'id': 'PVTF_lADOCY2ck84Aj7ZuzgcdIaw', 'name': 'Collaborators'}, {'id': 'PVTSSF_lADO

In [102]:
fields = response.json()['data']['node']['fields']['nodes']
#print(fields)
idx = next((index for (index,field) in enumerate(fields) if field['name'] == 'Repository name'),-1)
field_id = {field['name'] : field['id'] for field in fields[idx:]}
 
# For visibility (single-select field)
visib = [f for f in fields if f['name']=='Visibility'][0]
#print(visib)

visib_id = {}

visib_private = [f for f in visib['options'] if f['name'] == 'Private']
visib_id['Private'] = visib_private[0]['id']

visib_public = [f for f in visib['options'] if f['name'] == 'Public']
visib_id['Public'] = visib_public[0]['id']

#print
for n in field_id:
    print(f'{n}:\t {field_id[n]}')

print('\nVisibility Options:')
for i in visib_id:
    print(f'{i}:\t {visib_id[i]}')

Repository name:	 PVTF_lADOCY2ck84Aj7ZuzgcdIVo
WP:	 PVTF_lADOCY2ck84Aj7ZuzgcdIVs
Task:	 PVTF_lADOCY2ck84Aj7ZuzgcdIWg
Repository manager:	 PVTF_lADOCY2ck84Aj7ZuzgcdIZs
Collaborators:	 PVTF_lADOCY2ck84Aj7ZuzgcdIaw
Visibility:	 PVTSSF_lADOCY2ck84Aj7ZuzgcdIio

Visibility Options:
Private:	 8f5b6473
Public:	 169bd872


## Load the DataFrame

In [103]:
df_path = os.path.join(os.getcwd(),'Repository_list.csv')
df = pd.read_csv(df_path)
df

,Issue Number,Title,Node ID,Repository name,WP,Task,Repository manager,Collaborators,Private
0,10,[Repository Creation] - test_repo,I_kwDOMO4l_86OPOUC,test_repo,1,1.1,MclTTI,"[MclTTI, agalizia]",True
1,11,[Repository Creation] - test_repo_2,I_kwDOMO4l_86OPkVa,test_repo_2,1,1.3,MclTTI,"MclTTI, agalizia",True
2,13,[Repository Creation] - test_repo_3,I_kwDOMO4l_86OP2ml,test_repo_3,1,1.2,MclTTI,MclTTI,True


## Add items to the GitHub project

In [105]:
def add_item_query(project_id,issue_id):
  return f"""
  mutation {{
    addProjectV2ItemById(input: {{projectId: "{project_id}", contentId: "{issue_id}"}}) {{
      item {{
        id
      }}
    }}
  }}
  """



def update_field_query(project_id,item_id,field_id,value):

  #print('proj id: ',project_id)
  #print('item id: ',item_id)
  #print('field id: ',field_id)
  #print('value:',value)
  
  return f"""
  mutation {{
    updateProjectV2ItemFieldValue(input: {{
      projectId: "{project_id}",
      itemId: "{item_id}",  # Replace with the ID obtained from add_item_query
      fieldId: "{field_id}",
      value: {{
        text: "{value}"
      }}
    }}) {{
      projectV2Item {{
        id
      }}
    }}
  }}
  """



def set_single_select_field(project_id,item_id,field_id,option_id):

  #print('proj id: ',project_id)
  #print('item id: ',item_id)
  #print('field id: ',field_id)
  #print('option id:',option_id)
  
  return f"""
  mutation {{
    updateProjectV2ItemFieldValue(input: {{
      projectId: "{project_id}",
      itemId: "{item_id}",
      fieldId: "{field_id}",
      value: {{
        singleSelectOptionId: "{option_id}"
      }}
    }}) {{
      projectV2Item {{
        id
      }}
    }}
  }}
  """

In [117]:
def add_item(project_id,issue_id,field_id,visib_id):

    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Add new item to project
    response = requests.post(url, json={'query': add_item_query(project_id,issue_id)}, headers=headers)

    if response.status_code == 200:
        result = response.json()
        item_id = result['data']['addProjectV2ItemById']['item']['id']
        print(f'New item ID: {item_id}')

        # Update field of the new item
        for f in field_id:

            if f == 'Visibility':
            
                # Set visibility option ID
                if df.loc[df['Issue Number'] == 10,'Private'].item() == False:
                    visib = 'Public'
                else:
                    visib = 'Private'

                option_id = visib_id[visib]
            
                # Update visibility field
                print(f'Updating {f} <- {visib}')
                response = requests.post(url, json={'query': set_single_select_field(project_id,item_id,field_id[f],option_id)}, headers=headers)

            else:
                value = df.loc[df["Node ID"] == issue_id][f].item()
                print(f'Updating {f} <- {value}')
                response = requests.post(url, json={'query': update_field_query(project_id,item_id,field_id[f],value)}, headers=headers)

            if response.status_code == 200:
                pass
                #result = response.json()     
                #updated_item_id = result['data']['updateProjectV2ItemFieldValue']['projectV2Item']['id']
                #print('Updated field ID:', updated_item_id)
            else:
                print('Error in field update:', response.status_code, response.text)
        
    else:
        print('\nErrore in item add:', response.status_code, response.text)
    print()
    
    return None

### Config and run

In [119]:
print(f'Project ID: {project_id}\n')

for iss_num in df['Issue Number']:
    issue_id = df.loc[df['Issue Number'] == iss_num]['Node ID'].item()
    print(f'Issue Number {iss_num} ID: {issue_id}\n')
    add_item(project_id,issue_id,field_id,visib_id)
    print(f'Issue {iss_num} added successfully!\n')

Project ID: PVT_kwDOCY2ck84Aj7Zu

Issue Number 10 ID: I_kwDOMO4l_86OPOUC

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQnL-o
Updating Repository name <- test_repo
Updating WP <- 1
Updating Task <- 1.1
Updating Repository manager <- MclTTI
Updating Collaborators <- [MclTTI, agalizia]
Updating Visibility <- Private

Issue 10 added successfully!

Issue Number 11 ID: I_kwDOMO4l_86OPkVa

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQni4o
Updating Repository name <- test_repo_2
Updating WP <- 1
Updating Task <- 1.3
Updating Repository manager <- MclTTI
Updating Collaborators <- MclTTI, agalizia
Updating Visibility <- Private

Issue 11 added successfully!

Issue Number 13 ID: I_kwDOMO4l_86OP2ml

New item ID: PVTI_lADOCY2ck84Aj7ZuzgQni5E
Updating Repository name <- test_repo_3
Updating WP <- 1
Updating Task <- 1.2
Updating Repository manager <- MclTTI
Updating Collaborators <- MclTTI
Updating Visibility <- Private

Issue 13 added successfully!

